In [ ]:
import sys

ROOT_DIR = 'replace_with_your_root_directory'

sys.path.append(f'{ROOT_DIR}/MA_text2SQL/DataSampling/src/models/pipeline')

from pipeline.text2sql_pipeline import Text2SQLPipeline

BIRD_DATASET_PATH = f'{ROOT_DIR}/MA_text2SQL/DataSampling/src/src_data/bird_subset/stratified_output'
SPIDER_DATASET_PATH = f'{ROOT_DIR}/MA_text2SQL/DataSampling/src/src_data/spider_subset/spider_stratified_output_200'
API_KEY_PATH = f'{ROOT_DIR}/MA_text2SQL/Data/anthropic.api.key/text2sql.key'

# Load the API key
API_KEY = Text2SQLPipeline._get_together_api_key(API_KEY_PATH)

In [13]:
def get_output_response(response):
    thinking_messages = None
    response_messages = None
    
    for block in response.content:
        if block.type == "thinking":
            thinking_messages = block.thinking
        elif block.type == "redacted_thinking":
            thinking_messages = 'IT IS REDACTED'
        elif block.type == "text":
            response_messages = block.text

    return response_messages, thinking_messages

def count_tokens(client,messages):
    """Count tokens for a given message list."""
    result = client.messages.count_tokens(
        model="claude-3-7-sonnet-20250219",
        messages=messages
    )
    return result.input_tokens

In [17]:
from anthropic import Anthropic
from anthropic import NOT_GIVEN

client = Anthropic(api_key=API_KEY)

sample_prompt = """"What is the name of the bird species that has the longest wingspan?"""
system_prompt = """You are an animal expert."""

model_name = "claude-3-7-sonnet-20250219"

is_thinking = False

thinking={
        "type": "enabled",
        "budget_tokens": 2000,
    }

response = client.messages.create(
    model=model_name,
    system=system_prompt,
    thinking=thinking if is_thinking else NOT_GIVEN,
    messages=[
        {"role": "user", "content": sample_prompt},
    ],
    max_tokens= 4000, # It always should be higher than the budget tokens for thinking
)

response_messages, thinking_messages = get_output_response(response)
print("Thinking Messages: ", thinking_messages)
print(100 * "-")
print("Response Messages: ", response_messages)


Thinking Messages:  None
----------------------------------------------------------------------------------------------------
Response Messages:  The bird species with the longest wingspan is the Wandering Albatross (Diomedea exulans). These remarkable seabirds can have a wingspan measuring up to 3.5 meters (11.5 feet). They spend most of their lives soaring over the Southern Ocean surrounding Antarctica, using their enormous wings to ride air currents efficiently. Wandering Albatrosses can go years without touching land except when breeding, and they're capable of flying thousands of kilometers without flapping their wings, using a flying technique called dynamic soaring.


In [20]:
raw_response =  "<think>\nLet me break down what the user is asking for:\n\n1. Find accounts that have a loan with validity (duration) more than 24 months\n2. Among these accounts, find the ones with the lowest approved amount\n3. Filter to only include accounts that were opened before 1997\n4. List out these accounts\n\nLooking at the schema:\n- The `account` table has account information including account opening date (`date`)\n- The `loan` table has loan information including duration (`duration`) and amount (`amount`)\n\nSo I need to:\n1. Join the `account` and `loan` tables\n2. Filter for loans with duration > 24\n3. Filter for accounts opened before 1997-01-01\n4. Find the minimum approved amount\n5. Return the accounts that have this minimum amount\n\nLet me craft the SQL query:\n\n```sql\nWITH filtered_accounts AS (\n    SELECT a.account_id, a.date AS opening_date, l.amount\n    FROM account a\n    JOIN loan l ON a.account_id = l.account_id\n    WHERE l.duration > 24\n    AND a.date < '1997-01-01'\n),\nmin_amount AS (\n    SELECT MIN(amount) AS min_amount\n    FROM filtered_accounts\n)\nSELECT account_id, opening_date, amount\nFROM filtered_accounts\nWHERE amount = (SELECT min_amount FROM min_amount)\n```\n\nThis query:\n1. Creates a CTE that joins accounts and loans, filtering for accounts with loans > 24 months duration and opened before 1997\n2. Finds the minimum loan amount from this filtered set\n3. Returns all accounts from the filtered set that have this minimum amount\n\nLet me review it once more to ensure it's correct:\n\n- We're joining the account and loan tables correctly on account_id\n- We're filtering for loans with duration > 24 months\n- We're filtering for accounts opened before 1997-01-01\n- We're finding the minimum loan amount\n- We're returning accounts with that minimum amount\n\nThe query looks correct. I'll format it as the requested JSON response.\n</think>\n\nLooking at this request, I need to find accounts with loans over 24 months duration, with the lowest approved amount, and opened before 1997.\n\n```json\n{\n  \"sql\": \"WITH filtered_accounts AS (\n    SELECT a.account_id, a.date AS opening_date, l.amount\n    FROM account a\n    JOIN loan l ON a.account_id = l.account_id\n    WHERE l.duration > 24\n    AND a.date < '1997-01-01'\n)\nSELECT account_id, opening_date, amount\nFROM filtered_accounts\nWHERE amount = (\n    SELECT MIN(amount)\n    FROM filtered_accounts\n)\"\n}\n```"

print(raw_response)

<think>
Let me break down what the user is asking for:

1. Find accounts that have a loan with validity (duration) more than 24 months
2. Among these accounts, find the ones with the lowest approved amount
3. Filter to only include accounts that were opened before 1997
4. List out these accounts

Looking at the schema:
- The `account` table has account information including account opening date (`date`)
- The `loan` table has loan information including duration (`duration`) and amount (`amount`)

So I need to:
1. Join the `account` and `loan` tables
2. Filter for loans with duration > 24
3. Filter for accounts opened before 1997-01-01
4. Find the minimum approved amount
5. Return the accounts that have this minimum amount

Let me craft the SQL query:

```sql
WITH filtered_accounts AS (
    SELECT a.account_id, a.date AS opening_date, l.amount
    FROM account a
    JOIN loan l ON a.account_id = l.account_id
    WHERE l.duration > 24
    AND a.date < '1997-01-01'
),
min_amount AS (
    